### Create the dataset used for the visualisation


### Load the Space Catalogue as a dictionary


In [3]:
import urllib.request
satcat = "https://celestrak.com/pub/satcat.txt"
sat_catalogue = urllib.request.urlopen(satcat).read().decode('utf-8').strip().split("\n")

sat_dict = {}
for x in sat_catalogue:
    try:
        sat_dict[x.split()[1]] = x
    except IndexError:
        continue

In [4]:
import json
import csv
import datetime
from datetime import timedelta
import ephem

# Open the local TLE data file
local_tle_file = "update_tle.tsv"
output_file = "output/data.json"

# Read the local TLE data file
with open(local_tle_file, "r", encoding="utf-8") as file:
    reader = csv.reader(file, delimiter='\t')
    sheetValues = list(reader)


### Save the last coordinates when a satellites reenter the atmosphere

In [5]:
from sgp4.api import Satrec, jday
from datetime import timedelta, datetime
import numpy as np

data_list = []

for i in range(1, len(sheetValues)):
    row = sheetValues[i]
    tle_line0 = row[0]
    tle_line1 = row[1]
    norad_id = row[2].replace('n_', '')
    possible_satellite_reentry_date = sat_dict[norad_id][75:85] + " " + "00:00:00"
    
    try:
        # Initialize the satellite object with SGP4
        satellite = Satrec.twoline2rv(tle_line0, tle_line1)

        # Convert possible reentry date to datetime
        d = datetime.strptime(possible_satellite_reentry_date, '%Y-%m-%d %H:%M:%S')

        # Set reentry threshold to 120 km (120000 meters)
        reentry_threshold = 120  # in km

        # Propagate the TLE to find the reentry point
        reentered = False
        for hour_offset in range(0, 148):  # check up to 148 hours
            for minute_offset in range(0, 60, 1):  # check every minute
                current_time = d + timedelta(hours=hour_offset, minutes=minute_offset)
                jd, fr = jday(current_time.year, current_time.month, current_time.day,
                              current_time.hour, current_time.minute, current_time.second)
                e, r, v = satellite.sgp4(jd, fr)

                if e != 0:  
                    continue

                altitude = (r[0]**2 + r[1]**2 + r[2]**2)**0.5 - 6378.137  # Earth's radius in km

                if altitude <= reentry_threshold:
                    reentered = True
                    latitude = np.degrees(np.arctan2(r[2], (r[0]**2 + r[1]**2)**0.5))
                    longitude = np.degrees(np.arctan2(r[1], r[0]))

                    if np.isnan(latitude):
                        latitude = 'undefined'
                    if np.isnan(longitude):
                        longitude = 'undefined'

                    data_list.append({
                        "tle_0": tle_line0,
                        "tle_1": tle_line1,
                        "lat": latitude,
                        "lon": longitude,
                        "norad_cat_num": norad_id,
                        "satellite_name": sat_dict[norad_id][23:47].strip(),
                        "ownership": sat_dict[norad_id][49:54].strip(),
                        "launch_date": sat_dict[norad_id][56:66].strip(),
                        "satellite_decay": sat_dict[norad_id][75:85].strip(),
                        "satellite_decay_hour": str(current_time),
                        "rcs": sat_dict[norad_id][119:127].strip()
                    })

                    # Add further tracking points to show the reentry path
                    for offset_minutes in [4]: # [2, 6]
                        next_time = current_time + timedelta(minutes=offset_minutes)
                        jd, fr = jday(next_time.year, next_time.month, next_time.day,
                                      next_time.hour, next_time.minute, next_time.second)
                        e, r, v = satellite.sgp4(jd, fr)
                        if e != 0:  
                            continue

                        latitude = np.degrees(np.arctan2(r[2], (r[0]**2 + r[1]**2)**0.5))
                        longitude = np.degrees(np.arctan2(r[1], r[0]))

                        # Replace NaN with 'undefined'
                        if np.isnan(latitude):
                            latitude = 'undefined'
                        if np.isnan(longitude):
                            longitude = 'undefined'

                        data_list.append({
                            "tle_0": tle_line0,
                            "tle_1": tle_line1,
                            "lat": latitude,
                            "lon": longitude,
                            "norad_cat_num": norad_id,
                            "satellite_name": sat_dict[norad_id][23:47].strip(),
                            "ownership": sat_dict[norad_id][49:54].strip(),
                            "launch_date": sat_dict[norad_id][56:66].strip(),
                            "satellite_decay": sat_dict[norad_id][75:85].strip(),
                            "satellite_decay_hour": str(next_time),
                            "rcs": sat_dict[norad_id][119:127].strip()
                        })
                    break
            if reentered:
                break

    except Exception as e:
        print(norad_id, e)
        continue

print("\nFinished parsing!")

# Write data to minified JSON file
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump(data_list, json_file, ensure_ascii=False, separators=(',', ':'))


Finished parsing!
